## Semana 2 Día 2

**Modificaciones a tener en cuenta**

1. En lugar de usar SendGrid, he usado Gmail
2. He cambiado el nombre y la función de la empresa.
3. Las respuestas están al final del notebook

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
# import sendgrid # NO VOY A USAR SENDGRID, SINO GMAIL
import os
# from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart



In [32]:
load_dotenv(override=True)

True

In [ ]:
# Probando a enviar un email con Gmail

gmail_user = os.getenv("GMAIL_USER")
gmail_password = os.getenv("GMAIL_APP_PASSWORD")

def send_email(subject, body):
    to_list = ["junytrece@gmail.com", "musicaconyuri@gmail.com"]

    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = gmail_user
    msg['To'] = ", ".join(to_list)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(gmail_user, gmail_password)
        server.sendmail(gmail_user, to_list, msg.as_string())

send_email("Hola desde AI agent", "Este es un mensaje enviado por tu agente AI.")

## Paso 1: Flujo de trabajo del agente

In [ ]:
# Definiendo los system prompts para 3 agentes. Cada uno va a redactar un correo electrónico de ventas frío de una manera diferente.

instructions1 = "Eres un agente de ventas que trabaja para PlanchasChulas, \
    una empresa que vende planchas de última generación de cerámica\
    y otros productos para el cuidado del pelo. Redactas correos electrónicos en frío profesionales y serios."

instructions2 = "Eres un agente de ventas con sentido del humor y atractivo \
    que trabaja para PlanchasChulas, una empresa que vende planchas de última generación de cerámica \
    y otros productos para el cuidado del pelo. \
    Redactas correos electrónicos en frío ingeniosos y atractivos que probablemente obtengan respuesta."

instructions3 = "Eres un agente de ventas muy activo que trabaja para PlanchasChulas, \
    una empresa que vende planchas de última generación de cerámica\
    y otros productos para el cuidado del pelo. Redactas correos electrónicos en frío, concisos y directos."

In [ ]:
# Creando los agentes de ventas con las instrucciones indicadas para caada uno. 
# Estos agentes simulan diferentes enfoques de ventas, desde profesional hasta humorístico y directo.

sales_agent1 = Agent(
        name="Agente de ventas profesional",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Agente de ventas atractivo",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Agente de ventas ocpado",
        instructions=instructions3,
        model="gpt-4o-mini"
)

print("Los tres agentes de ventas han sido creados con éxito.")

In [35]:
# Definiendo el mensaje

nombre_empresa = "PlanchasChulas"
mi_nombre = "Ángela"
nombre_destinatario = "cliente"

message = f"Escribe un correo electrónico de ventas en frío. El nombre del destinatario es {nombre_destinatario}. \
    El nombre de la empresa es {nombre_empresa}. Mi nombre es {mi_nombre}."

In [36]:
# Ejecutando el agente de ventas profesional para redactar un correo electrónico (el primero)

result = Runner.run_streamed(sales_agent1, input=message)
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Asunto: Mejora la Experiencia de Cuidado del Cabello con PlanchasChulas

Estimado Cliente,

Espero que este mensaje le encuentre bien. Mi nombre es Ángela y formo parte del equipo de PlanchasChulas, una empresa dedicada a ofrecer planchas de última generación de cerámica y otros productos para el cuidado del cabello.

En un mercado donde la calidad y la innovación son fundamentales, nuestras planchas han sido diseñadas para proporcionar una experiencia única. Con tecnología avanzada y un acabado de cerámica que garantiza un deslizamiento suave, nuestras planchas no solo cuidan su cabello, sino que también ahorran tiempo en la rutina diaria de peinado.

Me encantaría explorar cómo PlanchasChulas puede ayudarle a mejorar su oferta de productos o a perfeccionar su experiencia de cuidado personal. ¿Podría agendar una llamada o una reunión breve para discutirlo? Estoy segura de que encontrará valor en nuestras propuestas.

Agradezco de antemano su atención y quedo a la espera de su respuest

In [12]:
# Ejecutando los agentes de ventas en paralelo para redactar correos electrónicos

# message = "Escribe un correo electrónico de ventas en frío" # Ya

with trace("Correos electrónicos fríos en paralelo"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Asunto: Mejora la rutina de cuidado de tu cabello con PlanchasChulas

Estimado Cliente,

Espero que este mensaje te encuentre bien. Mi nombre es Ángela y formo parte del equipo de PlanchasChulas, donde nos dedicamos a ofrecer productos innovadores para el cuidado del cabello, destacándonos especialmente por nuestras planchas de cerámica de última generación.

Sabemos lo importante que es mantener un cabello saludable y estilizado, y nuestras planchas están diseñadas para proporcionar un alisado perfecto sin dañar la estructura del pelo. Gracias a su tecnología avanzada, te garantizan un acabado profesional desde la comodidad de tu hogar.

Nos encantaría tener la oportunidad de presentarte nuestros productos y cómo pueden beneficiarte. Si estás interesado, estaré encantada de coordinar una breve llamada o enviarte más información sobre nuestras ofertas.

Agradezco tu atención y espero poder atender tus necesidades de cuidado capilar. 

Saludos cordiales,

Ángela  
PlanchasChulas  
[Tu n

In [ ]:
# Definiendo el agente de selección de correos electrónicos de ventas. Este agente va a seleccionar el mejor correo electrónico de ventas en frío entre las opciones disponibles.

sales_picker = Agent(
    name="sales_picker",
    instructions="Elige el mejor correo electrónico de ventas en frío entre las opciones disponibles. \
        Imagina que eres un cliente y elige el que probablemente te responda. \
        No des explicaciones; responde solo con el correo electrónico seleccionado.",
    model="gpt-4o-mini"
)

In [15]:
# message = "Escribe un correo electrónico de ventas en frío" # ya definido arriba

with trace("Selección del equipo de ventas"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Emails de ventas en frío:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"El mejor email de ventas:\n{best.final_output}")


El mejor email de ventas:
Asunto: ¡Descubre el poder de nuestras planchas de cerámica!

Hola Cliente,

Soy Ángela de PlanchasChulas. Quiero presentarte nuestras planchas de cerámica de última generación, diseñadas para ofrecerte un styling impecable y saludable. Con tecnología avanzada y resultados profesionales, lograrás el look que deseas en minutos.

Además, contamos con una variedad de productos para el cuidado del cabello que complementarán tu rutina de belleza.

¿Te gustaría conocer más? Estoy aquí para responder tus preguntas y ayudarte a encontrar la mejor opción para ti.

Saludos,  
Ángela  
PlanchasChulas  
[Tu número de contacto]  
[Tu correo electrónico]


Vamos a revisar ahora la traza:

https://platform.openai.com/traces

## Parte 2: Uso de herramientas

Ahora añadiremos una herramienta.

Recuerda todo el código JSON repetitivo y la función `handle_tool_calls()` con la lógica `if`.

In [ ]:
# Creando los tools, que no dejan de ser agentes...
# Esto ya estaba predefinido arriba, se repite para claridad

sales_agent1 = Agent(
        name="Agente de ventas profesional",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Agente de ventas atractivo",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Agente de ventas ocpado",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [10]:
sales_agent1

Agent(name='Agente de ventas profesional', instructions='Eres un agente de ventas que trabaja para ComplAI,     una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2    y prepararse para auditorías, impulsada por IA. Redactas correos electrónicos en frío profesionales y serios.', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Pasos 2 y 3: Interacción con herramientas y agentes

¿Recuerdas todo ese JSON repetitivo?

Simplemente encapsula tu función con el decorador `@function_tool`

In [ ]:
# USO GMAIL MÁS ABAJO, por eso no voy a usar SendGrid

# @function_tool
# def send_email(body: str):
#     """ Envía un correo electrónico con el cuerpo indicado a todos los clientes potenciales de ventas. """
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("juangabriel@frogames.es")  # Cambiar a tu remitente verificado
#     to_email = To("juangabriel@frogames.es")  # Cambiar a sureceptor
#     content = Content("text/plain", body)
#     mail = Mail(from_email, to_email, "Email de ventas", content).get()
#     response = sg.client.mail.send.post(request_body=mail)
#     return {"status": "success"}

In [ ]:
# Creando otra herramienta para enviar correos electrónicos. La diferencia aquí es que esta herramiento no es un agente, sino una función que envía un correo electrónico directamente.
# Esta herramienta se va a usar para enviar el correo electrónico de ventas seleccionado por el agente sales_picker

@function_tool
def send_email(subject: str, body: str):
    to_list = ["angeladiaz1997@gmail.com", "junytrece@gmail.com"]

    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = gmail_user
    msg['To'] = ", ".join(to_list)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(gmail_user, gmail_password)
        server.sendmail(gmail_user, to_list, msg.as_string())
    return {"status": "success"}

### Esto se ha convertido automáticamente en una herramienta, con el json repetitivo creado

In [22]:
# Echémosle un vistazo
send_email

FunctionTool(name='send_email', description='Envía un correo electrónico con el cuerpo indicado a todos los clientes potenciales de ventas.', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'body': {'title': 'Body', 'type': 'string'}, 'to': {'title': 'To', 'type': 'string'}}, 'required': ['subject', 'body', 'to'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020B1B586DE0>, strict_json_schema=True, is_enabled=True)

### Y también puedes convertir un Agente en una herramienta

In [18]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Escribe un email de ventas en frío")
tool1

FunctionTool(name='sales_agent1', description='Escribe un email de ventas en frío', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020B1CCB2200>, strict_json_schema=True, is_enabled=True)

### Ahora podemos reunir todas las herramientas:

Una herramienta para cada uno de nuestros tres agentes de redacción de correos electrónicos

Y una herramienta para nuestra función de envío de correos electrónicos

In [ ]:
# Reuniendo todos los tools en una lista, tanto los agentes convertidos en tool como la función de envío de correos electrónicos. Esto se usará en un agente final.

# description = "Escribe un correo electrónico de ventas en frío"
description = message  # Usando el mensaje definido arriba

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description="Envía un correo electrónico de ventas frío dirigido a 'Estimado cliente'", params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020B1F253560>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description="Envía un correo electrónico de ventas frío dirigido a 'Estimado cliente'", params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020B1F253CE0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description="Envía un c

## Y ahora es el momento para nuestro Gerente de Ventas - nuestro agente de planificación

In [ ]:
# Aquí definimos el agente de ventas final, que va a usar los tools definidos arriba para generar un correo electrónico de ventas en frío y enviarlo.

instructions ="Eres gerente de ventas y trabajas para PlanchasChulas. Tu nombre es Ángela. \
Utilizas las herramientas que te proporcionamos para generar correos electrónicos de ventas en frío. \
Nunca generas correos electrónicos de ventas tú mismo; siempre usas las herramientas. \
Pruebas las tres herramientas de sales_agent una vez antes de elegir la mejor. \
Eliges el mejor correo electrónico y usas la herramienta send_email para enviar el mejor correo electrónico (y solo el mejor) al usuario."


sales_manager = Agent(name="Manager de ventas", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Envía un correo electrónico de ventas frío dirigido a 'Estimado cliente'"

with trace("Manager de ventas"):
    result = await Runner.run(sales_manager, message)

## Recuerda revisar el seguimiento

https://platform.openai.com/traces

¡Y luego revisa tu correo electrónico!

### Las transferencias (handoffs) representan una forma en que un agente puede delegar en otro agente, transfiriéndole el control.

Las transferencias y los agentes como herramientas son similares:

En ambos casos, un agente puede colaborar con otro.

Con las herramientas, el control se transfiere.

Con las transferencias, el control se transfiere.

In [74]:
subject_instructions = "Puedes escribir un asunto para un correo electrónico de ventas en frío. \
    Se te proporciona un mensaje y necesitas escribir un asunto para un correo electrónico que probablemente obtenga respuesta."

html_instructions = "Puedes convertir un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML. \
    Se te proporciona un cuerpo de correo electrónico de texto que puede tener algún markdown \
    y necesitas convertirlo a un cuerpo de correo electrónico HTML con un diseño simple, claro y atractivo."

subject_writer = Agent(name="Escritor de asunto de correo electrónico", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", 
                                      tool_description="Escribe un asunto para un correo electrónico de ventas en frío")

html_converter = Agent(name="Conversor de cuerpo de correo electrónico HTML", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",
                                   tool_description="Convierte un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML")


In [ ]:
# USO GMAIL MÁS ABAJO, por eso no voy a usar SendGrid

# @function_tool
# def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
#     """ Envía un correo electrónico con el asunto y el cuerpo HTML a todos los clientes potenciales de ventas """
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("juangabriel@frogames.es")  # Cambiar a tu remitente verificado
#     to_email = To("juangabriel@frogames.es")  # Cambiar a sureceptor
#     content = Content("text/html", html_body)
#     mail = Mail(from_email, to_email, subject, content).get()
#     response = sg.client.mail.send.post(request_body=mail)
#     return {"status": "success"}

In [ ]:


@function_tool
def send_html_email(subject: str, html_body: str) -> dict:
    """
    Envía un correo electrónico con el asunto y el cuerpo HTML a todos los clientes potenciales de ventas usando Gmail.
    """
    gmail_user = os.getenv("GMAIL_USER")
    gmail_password = os.getenv("GMAIL_APP_PASSWORD")
    to_list = ["junytrece@gmail.com", "musicaconyuri@gmail.com"]

    msg = MIMEMultipart("alternative")
    msg['Subject'] = subject
    msg['From'] = gmail_user
    msg['To'] = ", ".join(to_list)
    msg.attach(MIMEText(html_body, "html"))

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(gmail_user, gmail_password)
        server.sendmail(gmail_user, to_list, msg.as_string())
    return {"status": "success"}

In [76]:
tools = [subject_tool, html_tool, send_html_email]

In [77]:
tools

[FunctionTool(name='subject_writer', description='Escribe un asunto para un correo electrónico de ventas en frío', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020B1F290C20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convierte un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020B1F290720>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', descr

In [ ]:
# Creando el agente de gestión de correos electrónicos que va a usar las herramientas definidas arriba para redactar un correo electrónico de ventas en frío, convertirlo a HTML y enviarlo.
# Este agente va a usar las herramientas subject_writer, html_converter y send_html_email para redactar, convertir y enviar el correo electrónico de ventas en frío.

instructions ="Eres un formateador y remitente de correos electrónicos. \
    Recibes el cuerpo de un correo electrónico para enviarlo. \
    Primero usas la herramienta subject_writer para escribir un asunto para el correo electrónico, \
    luego usas la herramienta html_converter para convertir el cuerpo a HTML. \
    Finalmente, usas la herramienta send_html_email para enviar el correo electrónico con el asunto y el cuerpo HTML."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convierte un email a HTML y lo envía")


### Ahora tenemos 3 herramientas y 1 transferencia

In [79]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description="Envía un correo electrónico de ventas frío dirigido a 'Estimado cliente'", params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020B1F253560>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description="Envía un correo electrónico de ventas frío dirigido a 'Estimado cliente'", params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020B1F253CE0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description="Envía un cor

In [ ]:
# Aquí definimos el agente de ventas final, que va a usar los tools definidos arriba para generar un correo electrónico de ventas en frío y enviarlo. La diferencia de este agente final
# con el que se difinió arriba está en que este agente va a usar las herramientas subject_writer, html_converter y send_html_email para redactar, convertir y enviar el correo electrónico de ventas en frío.
# El anterior no delegaba el envío del correo electrónico a otro agente, sino que lo hacía directamente, sin usar herramientas de HTML y subject.

sales_manager_instructions = "Eres un gerente de ventas que trabaja para PlanchasChulas. Te llamas Ángela. Utilizas las herramientas que se te proporcionan para generar correos electrónicos de ventas en frío. \
Nunca generas correos electrónicos de ventas tú mismo; siempre utilizas las herramientas. \
Pruebas las 3 herramientas del agente de ventas al menos una vez antes de elegir la mejor. \
Puedes usar las herramientas múltiples veces si no estás satisfecho con los resultados del primer intento. \
Seleccionas el mejor correo electrónico usando tu propio criterio sobre cuál será más efectivo. \
Después de elegir el correo electrónico, transfieres al agente Email Manager para formatear y enviar el correo."


sales_manager = Agent(
    name="Manager de ventas",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Envía un correo electrónico de ventas en frío dirigido a 'Estimado director ejecutivo'"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Recuerda revisar el seguimiento

https://platform.openai.com/traces

¡Y luego revisa tu correo electrónico!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Ejercicio</h2>
<span style="color:#ff7800;">¿Puedes identificar los patrones de diseño de Agentic que se han usado aquí?<br/>
¿Cuál es la línea que cambió esto de ser un "flujo de trabajo" de Agentic a un "agente" según la definición de Anthropic?<br/>
¡Intenta agregar más herramientas y agentes! Podrías tener herramientas que gestionen la combinación de correspondencia para enviar a una lista.<br/><br/>
RETO DIFÍCIL: Investiga cómo puedes hacer que SendGrid llame a un webhook de devolución de llamada cuando un usuario responde a un correo electrónico.
Luego, haz que el SDR responda para continuar la conversación. Esto puede requerir algo de programación de ambiente. 😂
</span>
        </td>
    </tr>
</table>

## Respuestas

1. Creo que es Orquestador-Trabajador. Hay un agente principal que es el **Manager de Ventas** el cual usa tres herramientas que a su vez no dejan de ser agentes convertidos en herramientas para escribir diferentes estilos de email. Y finalmente el agente el agente que manda emails aplicando html y añadiendo un asunto.
2. Creo que el momento clave es cuando dejamos al agente decidir cuál es el mejor correo. En ese momento se cumple la autonomía de un agente que decide una acción.